# Report Generation

## Overview
Automated generation of executive summaries and technical appendices.  Covers report configuration, validation, compilation, multi-format saving (Markdown / HTML), and batch scenario reports.

- **Prerequisites**: [reporting/01_table_generation](01_table_generation.ipynb)
- **Estimated runtime**: < 1 minute
- **Audience**: [Practitioner]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/reporting'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance.reporting.config import (
    ReportConfig, ReportMetadata, ReportStyle,
    SectionConfig, FigureConfig, TableConfig,
    create_executive_config, create_technical_config,
)
from ergodic_insurance.reporting.table_generator import TableGenerator
from ergodic_insurance.reporting.validator import ReportValidator, validate_results_data
from ergodic_insurance.reporting.executive_report import ExecutiveReport
from ergodic_insurance.reporting.technical_report import TechnicalReport

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Create Sample Analysis Results

Build the `results` and `parameters` dictionaries that the report system expects.

In [ ]:
n_sims, n_years = 1_000, 10
dt = 1 / 12
steps = n_years * 12
trajectories = np.zeros((n_sims, steps))
trajectories[:, 0] = 10_000_000
for i in range(1, steps):
    z = np.random.randn(n_sims)
    trajectories[:, i] = trajectories[:, i - 1] * (1 + 0.08 * dt + 0.15 * np.sqrt(dt) * z)

results = {
    "roe": 0.185, "roe_baseline": 0.155,
    "ruin_probability": 0.008, "ruin_probability_baseline": 0.025,
    "growth_rate": 0.072, "growth_rate_baseline": 0.065,
    "trajectories": trajectories,
    "optimal_limits": [5_000_000, 15_000_000, 10_000_000],
    "total_premium": 450_000, "expected_losses": 180_000,
    "frontier_data": {
        "ruin_probs": np.array([0.001, 0.005, 0.01, 0.02, 0.05, 0.1]),
        "roes": np.array([0.12, 0.15, 0.17, 0.19, 0.22, 0.25]),
        "optimal_point": (0.008, 0.185),
    },
    "convergence_metrics": {"gelman_rubin": 1.02, "ess": 5234, "autocorr": 0.045, "batch_p": 0.342},
    "sensitivity_analysis": {
        "Premium Rate": {"low": 0.16, "high": 0.21},
        "Loss Frequency": {"low": 0.17, "high": 0.19},
    },
    "simulated_losses": np.random.lognormal(10, 2, 1000),
    "correlation_matrix": np.corrcoef(np.random.randn(5, 100)),
}

parameters = {
    "financial": {"initial_assets": 10_000_000, "tax_rate": 0.25, "base_operating_margin": 0.08},
    "insurance": {"primary_limit": 5_000_000, "excess_limit": 15_000_000, "base_premium_rate": 0.03},
    "simulation": {"years": n_years, "num_simulations": n_sims, "seed": SEED},
}

print(f"Generated {n_sims} trajectories over {n_years} years")
print(f"ROE: {results['roe']:.1%}  |  Ruin: {results['ruin_probability']:.1%}")

## 2. Configure and Validate an Executive Report

In [ ]:
exec_config = ReportConfig(
    metadata=ReportMetadata(
        title="Insurance Optimization Analysis",
        subtitle="Executive Summary",
        authors=["Analytics Team"],
        date=datetime.now(),
        organization="Ergodic Insurance Analytics",
        confidentiality="Internal",
    ),
    style=ReportStyle(font_family="Arial", font_size=11, line_spacing=1.5),
    sections=[
        SectionConfig(title="Executive Summary", level=1,
                      content="Optimised insurance structuring delivers superior growth."),
        SectionConfig(title="Key Findings", level=1,
                      content="Risk-adjusted returns improve materially."),
        SectionConfig(title="Recommendations", level=1,
                      content="Implement the optimised structure immediately."),
    ],
    template="executive",
    output_formats=["markdown", "html"],
    output_dir=Path("reports"),
    cache_dir=Path("reports/cache"),
)

validator = ReportValidator(exec_config)
is_valid, errors, warns = validator.validate()
print(f"Config valid: {is_valid}  |  errors={len(errors)}  warnings={len(warns)}")

data_valid, data_errors = validate_results_data(results)
print(f"Results data valid: {data_valid}")

## 3. Compile and Preview Executive Report

In [ ]:
exec_report = ExecutiveReport(results=results, config=exec_config, cache_dir=Path("reports/cache"))
exec_content = exec_report.compile_report()

print(f"Report length: {len(exec_content):,} characters")
print("\n--- Preview (first 800 chars) ---")
print(exec_content[:800])

## 4. Technical Report

In [ ]:
tech_config = create_technical_config()
tech_report = TechnicalReport(
    results=results, parameters=parameters,
    config=tech_config, cache_dir=Path("reports/cache"),
)
tech_content = tech_report.compile_report()

print(f"Technical report: {len(tech_content):,} characters")
print(f"Validation metrics: {list(tech_report.validation_metrics.keys())}")

## 5. Save Reports

In [ ]:
exec_path = exec_report.save(output_format="markdown")
print(f"Executive (md)  : {exec_path}")

exec_html = exec_report.save(output_format="html")
print(f"Executive (html): {exec_html}")

tech_path = tech_report.save(output_format="markdown")
print(f"Technical (md)  : {tech_path}")

## 6. Batch Scenario Reports

In [ ]:
scenarios = [
    {"name": "Conservative", "roe": 0.14, "ruin_prob": 0.001},
    {"name": "Balanced",     "roe": 0.18, "ruin_prob": 0.008},
    {"name": "Aggressive",   "roe": 0.22, "ruin_prob": 0.025},
]

for sc in scenarios:
    sc_results = results.copy()
    sc_results["roe"] = sc["roe"]
    sc_results["ruin_probability"] = sc["ruin_prob"]
    sc_config = create_executive_config()
    sc_config.metadata.title = f"Insurance Analysis -- {sc['name']}"
    report = ExecutiveReport(results=sc_results, config=sc_config)
    path = report.save(output_format="markdown")
    print(f"{sc['name']:>14s}  ROE={sc['roe']:.1%}  Ruin={sc['ruin_prob']:.1%}  -> {path}")

## Key Takeaways

- `ReportConfig` + `ReportValidator` enforce structure before generation.
- `ExecutiveReport` produces concise summaries; `TechnicalReport` adds methodology and validation details.
- Reports can be saved as Markdown or HTML and batch-generated for multiple scenarios.

## Next Steps

- [reporting/01_table_generation](01_table_generation.ipynb) -- table formatting utilities
- [reporting/02_excel_reporting](02_excel_reporting.ipynb) -- Excel financial statements